## This file is used to preprocess some data we will use later

In [1]:
import torch
import os
import pandas as pd
import numpy as np
import time
import zipfile


In [2]:
# we first get all sentences data and put them together
sentences_data_path_zipped = '/data/Dropbox/Data/Circuit_Courts/circuit-cases/sentences.zip'
processed_data_path = '/data/Dropbox/judge_embedding_data_sp18'
sentences_data_path = '/data/Dropbox/judge_embedding_data_sp18/sentences_new'


In [10]:
def convert_to_binary_sentence_data(data_folder_path,processed_data_folder_path,
                                    data_binary_name="sentence_data", 
                                    data_count_limit = 999999999,verbose=1):
    # data count limit is how many txt files do you want in total
    # data dump limit is every how many txt files do you want to do a binary dump
    start_time = time.time()
    # for test purposes, data limit can be set to indicate how much data to use
    data_count = 0
    # give the circuit court main folder's path, read all data
    folder_names = os.listdir(data_folder_path)
    folder_names.sort()
    
#   judge_df = pandas.DataFrame(columns=["Judge_Name","Year","Sentence"])
    data_list = []
    save_part_index = 0
    
    finished = False
    
    for folder_name in folder_names: # for each folder
        if verbose > 0:
            print("now process:",folder_name,"current data count:",data_count,"time used:",time.time()-start_time)
        
        year = folder_name[-4:]
        data_file_names = os.listdir(os.path.join(data_folder_path,folder_name))
        
        if finished:
            break
        
        for file_name in data_file_names: # for each file
            file_name_without_txt = file_name[:-4]
            file_name_tokens = file_name_without_txt.split("_")
            
            if len(file_name_tokens)<3:
                print("file format incorrect at file:",file_name)
                continue
            
            caseid = file_name_tokens[0]
            middle_part =file_name_tokens[1]
            judge_name = file_name_tokens[2] # we get the judge's name from the file name
            
            file_path = os.path.join(data_folder_path,folder_name,file_name)
            fpt = open(file_path,"r")
            sentence = fpt.read()
            fpt.close()
            
            new_data_entry = [caseid,year,judge_name,middle_part,sentence]
            data_list.append(new_data_entry)
            
            data_count += 1
            
            if data_count > data_count_limit: # for debugging purposes
                finished = True
                break
    
    df = pd.DataFrame(data_list,columns = ["caseid","year","judge_last_name","middle_part","opinion_text"])
        
    return df

In [11]:
# total data is around 450,000, of total size about 6GB
df_ready_to_csv = convert_to_binary_sentence_data(sentences_data_path,processed_data_path,verbose=1)

# convert_to_binary_sentence_data(sentence_folder_path,processed_data_folder_path,verbose=1)

now process: sent_1891 current data count: 0 time used: 0.0009055137634277344
now process: sent_1892 current data count: 26 time used: 0.006361246109008789
now process: sent_1893 current data count: 291 time used: 2.4946911334991455
now process: sent_1894 current data count: 750 time used: 6.348055839538574
now process: sent_1895 current data count: 1220 time used: 10.731652975082397
now process: sent_1896 current data count: 1813 time used: 15.936142921447754
now process: sent_1897 current data count: 2434 time used: 21.18468976020813
now process: sent_1898 current data count: 2991 time used: 26.363637447357178
now process: sent_1899 current data count: 3579 time used: 31.338996171951294
now process: sent_1900 current data count: 4214 time used: 36.5309898853302
now process: sent_1901 current data count: 4883 time used: 40.961482763290405
now process: sent_1902 current data count: 5571 time used: 46.32253360748291
now process: sent_1903 current data count: 6296 time used: 51.556399345

now process: sent_1996 current data count: 321149 time used: 2619.3874554634094
now process: sent_1997 current data count: 328478 time used: 2714.3940827846527
now process: sent_1998 current data count: 335258 time used: 2791.442756175995
now process: sent_1999 current data count: 341846 time used: 2895.940526008606
now process: sent_2000 current data count: 348483 time used: 3007.429242372513
now process: sent_2001 current data count: 355026 time used: 3103.624356031418
now process: sent_2002 current data count: 362292 time used: 3206.4510831832886
now process: sent_2003 current data count: 368648 time used: 3300.75541806221
now process: sent_2004 current data count: 375020 time used: 3388.165833234787
now process: sent_2005 current data count: 381365 time used: 3526.765026807785
now process: sent_2006 current data count: 387426 time used: 3610.4052772521973
now process: sent_2007 current data count: 393208 time used: 3681.389084815979
now process: sent_2008 current data count: 398708

In [12]:
df_ready_to_csv.head()

,caseid,year,judge_last_name,middle_part,opinion_text
0,XB0PMNQNB5G0,1891,PARDEE,contentMajOp,"P ARDEE , J., ( after stating the case .)\nTh..."
1,XFL757,1891,BROWN,contentMajOp,"B ROWN , Circuit Justice, ( orally .)\nIn thi..."
2,XB0PMRQNB5G0,1891,PARDEE,contentMajOp,"P ARDEE , J. The case on this intervention is..."
3,XFL7KI,1891,LOCKE,contentMajOp,"L OCKE , District Judge.\nThis motion coming ..."
4,XB0OI7QNB5G0,1891,NELSON,contentMajOp,"N ELSON , J. This was an action brought to re..."


In [ ]:
#Now we can save these data as csv files